In [4]:
# logger
import logging
import sys

def setup_logger():
    logging.basicConfig(level=logging.INFO,
                        format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
                        handlers=[logging.StreamHandler(sys.stdout)])


In [5]:
# fake database

from dataclasses import dataclass
from typing import Dict

logger = logging.getLogger("repository")

@dataclass
class User:
    id: int
    email: str
    is_active: bool = True

class Repository:
    def __init__(self) -> None:
        self._users: Dict[int, User] = {}
    
    def save(self, user: User):
        logger.info("saving user id=%s", user.id)
        self._users[user.id] = user
    
    def get(self, user_id:int):
        logger.info("fetching user id=%s", user_id)
        return self._users.get(user_id)
    


In [6]:
logger2 = logging.getLogger("service")

class UserService:
    def deactivate_user(self, user: User):
        logger.info("deactivating user id=%s", user.id)

        if not user.is_active:
            logger2.warning("user already inactive")
            return user
        
        return User(id = user.id,
                    email=user.email,
                    is_active=False,)

In [7]:
logger3 = logging.getLogger("app")

class UserApplication:
    def __init__(self, repo: Repository, service: UserService) -> None:
        self.repo = repo
        self.service = service

    def handle_deactivate(self, user_id: int):
        logger3.info("received deactivate req for user id=%s", user_id)

        user = self.repo.get(user_id=user_id)
        if user is None:
            logger3.error("user not found")
            return
        
        updated_user = self.service.deactivate_user(user)
        self.repo.save(updated_user)

        logger3.info("user %s is active=%s", updated_user.id, updated_user.is_active,)

In [9]:
def main():
    setup_logger()

    repo = Repository()
    service = UserService()
    app = UserApplication(repo=repo, service=service)

    repo.save(User(1, "batman@gmail.com"))
    repo.save(User(1, "joker@gmail.com", is_active=False))

    print("\n Simulated Requests \n")
    app.handle_deactivate(1)
    print()
    app.handle_deactivate(100)
    print()
    app.handle_deactivate(2)


main()


2025-12-26 20:24:27,943 | INFO | repository | saving user id=1
2025-12-26 20:24:27,945 | INFO | repository | saving user id=1

 Simulated Requests 

2025-12-26 20:24:27,945 | INFO | app | received deactivate req for user id=1
2025-12-26 20:24:27,946 | INFO | repository | fetching user id=1
2025-12-26 20:24:27,947 | INFO | repository | deactivating user id=1
2025-12-26 20:24:27,947 | WARNING | service | user already inactive
2025-12-26 20:24:27,948 | INFO | repository | saving user id=1
2025-12-26 20:24:27,948 | INFO | app | user 1 is active=False

2025-12-26 20:24:27,949 | INFO | app | received deactivate req for user id=100
2025-12-26 20:24:27,949 | INFO | repository | fetching user id=100
2025-12-26 20:24:27,950 | ERROR | app | user not found

2025-12-26 20:24:27,951 | INFO | app | received deactivate req for user id=2
2025-12-26 20:24:27,951 | INFO | repository | fetching user id=2
2025-12-26 20:24:27,952 | ERROR | app | user not found
